In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
import time
from splinter import Browser

### NASA Mars News  
#### Collect the latest title and paragraph 
#### Assign text to variables


In [2]:
local_nasa_file = "News_NASA_Mars_Exploration_Program.html"

In [3]:
nasa_html = open(local_nasa_file, "r").read()

In [4]:
news_soup = bs(nasa_html, "html.parser")
# print(news_soup)

In [5]:
news_title = news_soup.title.text
print(news_title)

News  – NASA’s Mars Exploration Program 


In [6]:
news_p = news_soup.body.find('p').text
print(news_p)

Managed by the Mars Exploration Program and the Jet Propulsion Laboratory for NASA’s Science Mission Directorate


In [7]:
# print(type(news_soup.body.find('p')))
# print('\n')
# print(news_soup.body.find('p'))

### JPL Mars Space Images - Featured Image
#### Find the image url for the current Featured Mars Image
#### Assign the url string to a variable 

In [8]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
rs = requests.get(url)
image_soup = bs(rs.text, 'html.parser')
image = image_soup.find("img", class_="thumb")["src"]
featured_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars" + image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA22883-640x350.jpg


### Mars Weather
#### Scrape the latest tweet and save the tweet text as a variable

In [9]:
url = 'https://twitter.com/marswxreport?lang=en'

r = requests.get(url)
soup = bs(r.text, 'html.parser')
mars_tweets = soup.find('ol', class_='stream-items')
mars_weather = mars_tweets.find('p', class_="tweet-text").text
print(mars_weather)

Well done! That 30 minutes of EDL dust settling was very effective. Shame #InSight can’t act as a supercharger for @marsrovers Oppy, she sure could use a boost right now.https://twitter.com/NASAInSight/status/1068661716756516864 …


### Mars Facts
#### use pandas to scrape the table and convert to html


In [10]:
url = 'https://space-facts.com/mars/'

In [11]:
tables = pd.read_html(url)

In [16]:
df = tables[0]

In [17]:
# df.columns = ["Parameter", "Value"]

In [18]:
# df.set_index(["Parameter"])

In [19]:
html_table = df.to_html()

In [20]:
df.to_html('table.html')

In [21]:
!open table.html

### Mars Hemispheres
#### Visit USGS astrogeology site 
#### Save the hemisphere title and img-url
#### Append the dictionary to a list

In [22]:
hemisphere_image_urls = [{"title": "Valles Marineris Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg"},
    
                        {"title": "Cerberus Hemisphere","img_url":"https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg "},
                        
                        {"title": "Schiaparelli Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg"},
                        {"title": "Syrtis Major Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg"},
                        
                        ]
hemisphere_image_urls

[{'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'},
 {'title': 'Cerberus Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg '},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}]

In [28]:


# Create connection variable
conn = 'mongodb://localhost:27017'

# Pass connection to the pymongo instance.
client = pymongo.MongoClient(conn)


In [29]:
db = client.mars_db

In [30]:
mars_info = db.mars_info

In [31]:
record = {
    "Featured_image": featured_image_url,
    "Hemisphere": hemisphere_image_urls,
    "Title": news_title,
    "Paragraph": news_p, 
    "weather": mars_weather,
    "Mars_Facts": html_table
}

In [32]:
mars_info.insert_one(record)